# Spark DataFrame

## Spark DataFrames GroupBy and Aggregate

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('aggs').getOrCreate()

In [3]:
df = spark.read.csv('sales_info.csv', inferSchema=True, header=True)

In [4]:
df.show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|    Sam|200.0|
|   GOOG|Charlie|120.0|
|   GOOG|  Frank|340.0|
|   MSFT|   Tina|600.0|
|   MSFT|    Amy|124.0|
|   MSFT|Vanessa|243.0|
|     FB|   Carl|870.0|
|     FB|  Sarah|350.0|
|   APPL|   John|250.0|
|   APPL|  Linda|130.0|
|   APPL|   Mike|750.0|
|   APPL|  Chris|350.0|
+-------+-------+-----+



In [5]:
df.printSchema()

root
 |-- Company: string (nullable = true)
 |-- Person: string (nullable = true)
 |-- Sales: double (nullable = true)



In [6]:
df.groupBy('Company')
# 그룹화된 데이터 오브젝트 그룹이 반환된다. 메모리에서의 위치를 보여준다.

그룹화된 데이터 오브젝트에 사용가능한 함수에 대한 정보 링크 <br>
http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#module-pyspark.sql.functions <br>
이전 버전의 강의라 없어진 페이지가 나옴

In [8]:
df.groupBy('Company').mean()

DataFrame[Company: string, avg(Sales): double]

In [9]:
df.groupBy('Company').mean().show()
# sum/max/min/count

+-------+-----------------+
|Company|       avg(Sales)|
+-------+-----------------+
|   APPL|            370.0|
|   GOOG|            220.0|
|     FB|            610.0|
|   MSFT|322.3333333333333|
+-------+-----------------+



In [10]:
# 모든 method를 groupby로 불러올 필요는 없다.
# agg method를 가지고 모든 행들을 데이터프레임에 집합시킬 수 있다.

# 그룹화 시키지 않고 단지 매출의 총합을 구하고 싶을 수 있다.
# 이런 경우 흥미로운 인자를 가져오는데 dictionary를 가져온다.
df.agg({'Sales':'sum'}).show()
# 출력: Sales 컬럼의 sum값이 출력된다.

+----------+
|sum(Sales)|
+----------+
|    4327.0|
+----------+



In [11]:
# 물론 groupBy와 agg를 함께 사용할 수 있다.
df.groupBy('Company').agg({'Sales':'max'}).show()
# df.groupBy('Company').max().show()와 같은 결과이다.
# 반복문을 사용하는 경우 더 좋다고 한다.

+-------+----------+
|Company|max(Sales)|
+-------+----------+
|   APPL|     750.0|
|   GOOG|     340.0|
|     FB|     870.0|
|   MSFT|     600.0|
+-------+----------+



In [12]:
# 어떻게 스파크에서 함수를 불러오는지 보자.
# (다른 사용법을 보자는 의미이다.)
from pyspark.sql.functions import avg, stddev

In [13]:
df.select(avg('Sales')).show()
# df.agg({'Sales':'avg'}).show()와 같은 결과이다.
# 하지만 더 보편적인 방법이다.
# 함수를 불러와 원하는 컬럼에 적용한다.
# 그리고 그 결과를 show()/collect()할 수 있다.

+-----------------+
|       avg(Sales)|
+-----------------+
|360.5833333333333|
+-----------------+



In [30]:
# 참고!
avg('Sales')

Column<'avg(Sales)'>

In [14]:
# 별칭 사용
df.select(avg('Sales').alias('Average Sales')).show()

+-----------------+
|    Average Sales|
+-----------------+
|360.5833333333333|
+-----------------+



In [15]:
# 출력 format 조정 - format_number
from pyspark.sql.functions import format_number

In [17]:
sales_std = df.select(stddev('Sales').alias('std'))
sales_std.show()

+------------------+
|               std|
+------------------+
|250.08742410799007|
+------------------+



In [18]:
sales_std.select(format_number('std',2)).show()
# format_number( , 2)는 소수점을 설정

+---------------------+
|format_number(std, 2)|
+---------------------+
|               250.09|
+---------------------+



In [19]:
sales_std.select(format_number('std',2).alias('std')).show()
# 컬럼명이 지저분해 alias를 적용하였다.

+------+
|   std|
+------+
|250.09|
+------+



In [33]:
# 위 과정을 한 줄로
df.select(format_number(stddev('Sales'),2).alias('std')).show()

+------+
|   std|
+------+
|250.09|
+------+



### 정리
- groupBy
> df.groupBy('컬럼이름').그룹함수() <br>
> df.groupBy('컬럼이름').agg({'컬럼이름':'집계함수')

- 함수사용
> df.agg({'컬럼이름':'함수'}) <br>
> df.select(함수('컬럼이름'))

In [20]:
# 정렬
# 오름차순
df.orderBy('Sales').show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|Charlie|120.0|
|   MSFT|    Amy|124.0|
|   APPL|  Linda|130.0|
|   GOOG|    Sam|200.0|
|   MSFT|Vanessa|243.0|
|   APPL|   John|250.0|
|   GOOG|  Frank|340.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   MSFT|   Tina|600.0|
|   APPL|   Mike|750.0|
|     FB|   Carl|870.0|
+-------+-------+-----+



In [21]:
# 내림차순
df.orderBy(df['Sales'].desc()).show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|     FB|   Carl|870.0|
|   APPL|   Mike|750.0|
|   MSFT|   Tina|600.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   GOOG|  Frank|340.0|
|   APPL|   John|250.0|
|   MSFT|Vanessa|243.0|
|   GOOG|    Sam|200.0|
|   APPL|  Linda|130.0|
|   MSFT|    Amy|124.0|
|   GOOG|Charlie|120.0|
+-------+-------+-----+



In [22]:
# 참고!
df['Sales'].desc()

Column<'Sales DESC NULLS LAST'>